# Pandas Test - Real life Exercise

In [1]:
import pandas as pd
import math
from scipy.stats import norm

For visualizations:

In [2]:
import cufflinks as cf; cf.go_offline()
import plotly_express as px

## About the data

The data you are about to load is coming from an Ad recommentation system of the company InBrain. InBrain is a adTech company and specalize in next-level ad recommendation.  
The company has two major products - one giving high quality recommendation while the other is giving an highly yielding recommendation. Inbrain custormers are sending requests, asking for either of the products. 

Once a week, the company is doing an internal quality assements, and sends a sample of its traffic requests to external annotation process, in which the annotators are trained to manually decided whether the recommended Ad is a succes or not. The data contains information about the ad (**ad_id**), the sampled week (**week_id**), the requester company (**requester**), the region of operation (**region**), the recommendation type (Quality/Yield, **recommendation_type**) and the recommendation provider algorithm (**rec_provider**). The annotators are marking whether the recommendation is meeting the Quality/Yield standards (**is_success**) and whether or not the recommendation is severly defected (**is_sever**)

See a sample below:

In [3]:
ad_recs_annotated = pd.read_csv('data/ad_annotations.csv')
df=ad_recs_annotated
df.sample(5)

,week_id,ad_id,requester,region,is_success,is_sever,recommendation_type,rec_provider
144623,2021-week_43,AD078SV88K,Fry Multimedia,DE,yes,NaN,Qality,UserPopQ
29007,2021-week_35,AD88998078,Search Million Culture,DE,yes,NaN,Yield,BooksY
293447,2022-week_02,AD0789S8J8,RelayFoods,IN,yes,NaN,Qality,UserPopQ
119978,2021-week_41,AD00F8X8SU,Modanisa,GB,no,True,Yield,RuleBasedY
39144,2021-week_35,AD08PZ879M,Sensor Tower,US,yes,NaN,Qality,DNNQ


Your job, as the new and only data scientist of the company, is to get familiar with the the data, show week-over-week trends and produce insightfull graphs as a preparation to a full blown BI dashboard.

## Questions

### Data Modifications

1. Add a column with the sample year
1. Add a column with the sample quarter (google how to)
1. Add a new success column with values 1 and 0 for yes and no 
1. Are there any NaNs in the is_sever column? Count how many and create a new column with NaNs filled as False. Check that indeed the new column contaion no NaNs. 
1. Capitlize (first letter only) the is_success column (Inplace)

#### A to Q1:

In [4]:
df['year'] = df.week_id.apply(lambda w: w.split('-')[0])

#### A to Q2:

In [5]:
df['quarter'] = df.week_id.apply(lambda w: int(w.split('_')[1]) // 13 + 1)

#### A to Q3:

In [6]:
df['is_success_int'] = df.is_success.map({'yes': 1, 'no': 0})
df.sample(5)

,week_id,ad_id,requester,region,is_success,is_sever,recommendation_type,rec_provider,year,quarter,is_success_int
151525,2021-week_43,AD08FDIJD0,Puentes Company,IN,yes,NaN,Yield,UserPopY,2021,4,1
177128,2021-week_45,AD08LWAUW8,Search Million Culture,GB,yes,NaN,Qality,RNNQ,2021,4,1
269451,2021-week_51,AD088PN899,Extreme DA,US,yes,NaN,Yield,RuleBasedY,2021,4,1
175345,2021-week_45,AD07L8LLRS,Fancy,GB,yes,NaN,Qality,DNNQ,2021,4,1
305246,2022-week_03,AD08MY8P8F,RelayFoods,GB,yes,NaN,Qality,RNNQ,2022,1,1


#### A to Q4:

In [7]:
df.is_sever.isnull().sum() # Yes, we have NaNs 

304195

In [8]:
df[df.is_sever.notnull()].is_success.value_counts()

no     69576
yes       17
Name: is_success, dtype: int64

In [9]:
df['is_sever_fixed'] = df.is_sever.fillna(False)
df['is_sever_fixed'].isnull().sum()

0

In [10]:
df.is_sever.value_counts(dropna=False)

NaN      304195
True      44689
False     24904
Name: is_sever, dtype: int64

### Subset selection

for each question, sub-select the data by using the conditional selection (`[]`) and the `.query` API. Use `.shape` on the subselection to obtain the result.  
For example:
`df.query('some_condition').shape`

1. How many requests are there in US region? 
1. How many **successful** requests are there in US region? 
1. How many **successful** requests are there in US region, on week 52?
1. How many requests where delivered by DDNQ, RRNY and UserPopQ together? 
1. How many requests where delivered by rule based providers? 
1. Select only the requests from 2021 - How many are there? 
1. Select the reqeusts from week 30 to week 40 - How many are there? 
1. Select all the data that comes from the most popular Ad
1. Select all the data that comes from the least popular provider
1. Select the data in which is_sever is None. How big is it? 
1. Select the data in which the requester is a 3 word company
1. Select the data in which the requester is a 3 word company, and the ad_id has the letter 6 in it
1. Select the data in which the requester is a 3 word company, and the multiplication of all the numbers in the ad_id is bigger than 30

#### A to Q1:

In [11]:
print(df[df.region == "US"].shape[0])
print(df.query("region == 'US'").shape[0])

103846
103846


#### A to Q2:

In [12]:
print(df[(df.region == "US") & (df.is_success == "yes")].shape[0])
print(df.query('region == "US" and is_success == "yes"').shape[0])

88918
88918


#### A to Q4:

In [13]:
print(df[(df.rec_provider=='DDNQ') | (df.rec_provider=='RRNY') | (df.rec_provider=='UserPopQ')].shape[0])
print(df.query("rec_provider=='DDNQ'| rec_provider=='RRNY' | rec_provider=='UserPopQ'").shape[0])

69937
69937


#### A to Q5:

In [14]:
df[df.rec_provider.str[0:9]=='RuleBased'].shape[0]

28336

#### A to Q6:

In [15]:
df[df.year.astype(int)>=2021].requester.count()

373788

#### A to Q7:

In [16]:
week_number=df.week_id.apply(lambda w: int(w.split('_')[1])) 
df[(week_number>=30) & (week_number<=40)]
# df.query('week_number>=30 & week_number<=40')

,week_id,ad_id,requester,region,is_success,is_sever,recommendation_type,rec_provider,year,quarter,is_success_int,is_sever_fixed
0,2021-week_33,AD0088VOS,Search Million Culture,DE,yes,NaN,Qality,DNNQ,2021,3,1,False
1,2021-week_33,AD07KYS8JM,MoJoe Brewing Company,DE,yes,NaN,Yield,DNNY,2021,3,1,False
2,2021-week_33,AD08PDP6Y9,Search Million Culture,DE,yes,NaN,Qality,DNNQ,2021,3,1,False
3,2021-week_33,AD89608808,Search Million Culture,DE,yes,NaN,Qality,BooksQ,2021,3,1,False
4,2021-week_33,AD07CMVHP6,Search Million Culture,DE,yes,NaN,Qality,DNNQ,2021,3,1,False
...,...,...,...,...,...,...,...,...,...,...,...,...
115046,2021-week_40,AD07XH8N9W,Sensor Tower,US,yes,NaN,Qality,DNNQ,2021,4,1,False
115047,2021-week_40,AD097JK9F8,Extreme DA,US,yes,NaN,Yield,RNNY,2021,4,1,False
115048,2021-week_40,AD086WDR8S,Extreme DA,US,yes,NaN,Yield,DNNY,2021,4,1,False
115049,2021-week_40,AD07J897RY,Sensor Tower,US,yes,NaN,Yield,DNNY,2021,4,1,False


#### A to Q8:

In [17]:
df[df.ad_id==df.ad_id.value_counts().idxmax()]
# df.query('ad_id == ad_id.value_counts().idxmax()')

,week_id,ad_id,requester,region,is_success,is_sever,recommendation_type,rec_provider,year,quarter,is_success_int,is_sever_fixed
7031,2021-week_33,AD07PFFMP9,Puentes Company,IN,yes,NaN,Yield,ManualQ,2021,3,1,False
7032,2021-week_33,AD07PFFMP9,Puentes Company,IN,yes,NaN,Yield,ManualQ,2021,3,1,False
20010,2021-week_34,AD07PFFMP9,Bizanga,IN,yes,NaN,Yield,ManualQ,2021,3,1,False
20011,2021-week_34,AD07PFFMP9,Puentes Company,IN,yes,NaN,Yield,ManualQ,2021,3,1,False
35214,2021-week_35,AD07PFFMP9,Bizanga,IN,yes,NaN,Yield,ManualQ,2021,3,1,False
...,...,...,...,...,...,...,...,...,...,...,...,...
363533,2022-week_52,AD07PFFMP9,MoJoe Brewing Company,IN,yes,NaN,Yield,ManualQ,2022,5,1,False
363534,2022-week_52,AD07PFFMP9,MoJoe Brewing Company,IN,yes,NaN,Yield,ManualQ,2022,5,1,False
363535,2022-week_52,AD07PFFMP9,Bizanga,IN,yes,NaN,Yield,ManualQ,2022,5,1,False
363536,2022-week_52,AD07PFFMP9,MoJoe Brewing Company,IN,yes,NaN,Yield,ManualQ,2022,5,1,False


#### A to Q9:

In [18]:
df[df.rec_provider==df.rec_provider.value_counts().idxmin()]
# df.query('rec_provider == rec_provider.value_counts().idxmin()')

,week_id,ad_id,requester,region,is_success,is_sever,recommendation_type,rec_provider,year,quarter,is_success_int,is_sever_fixed
352263,2022-week_06,AD0088UP8R,MoJoe Brewing Company,JP,yes,NaN,Yield,UserPopSelectionY,2022,1,1,False
352326,2022-week_06,AD09C8KC8Z,MoJoe Brewing Company,JP,yes,NaN,Yield,UserPopSelectionY,2022,1,1,False
352578,2022-week_06,AD078DT6GJ,MoJoe Brewing Company,JP,yes,NaN,Yield,UserPopSelectionY,2022,1,1,False
352595,2022-week_06,AD09C8P789,MoJoe Brewing Company,JP,yes,NaN,Yield,UserPopSelectionY,2022,1,1,False
352703,2022-week_06,AD00UF8VOF,MoJoe Brewing Company,JP,yes,NaN,Yield,UserPopSelectionY,2022,1,1,False
352714,2022-week_06,AD098CXHRF,MoJoe Brewing Company,JP,yes,NaN,Yield,UserPopSelectionY,2022,1,1,False
352738,2022-week_06,AD0086GLW0,MoJoe Brewing Company,JP,yes,NaN,Yield,UserPopSelectionY,2022,1,1,False
352740,2022-week_06,AD08QMJ888,MoJoe Brewing Company,JP,yes,NaN,Yield,UserPopSelectionY,2022,1,1,False
352745,2022-week_06,AD07H9GM98,MoJoe Brewing Company,JP,yes,NaN,Yield,UserPopSelectionY,2022,1,1,False
352803,2022-week_06,AD09G9YKC8,MoJoe Brewing Company,JP,no,False,Yield,UserPopSelectionY,2022,1,0,False


#### A to Q10:

In [19]:
df[df.is_sever.isnull()].shape[0]
# df.query('is_sever.isnull()').shape[0]

304195

#### A to Q11:

In [20]:
three_words_requesters = df[df.requester.apply(lambda s: len(s.split(' ')) == 3)]
three_words_requesters.sample(5)

,week_id,ad_id,requester,region,is_success,is_sever,recommendation_type,rec_provider,year,quarter,is_success_int,is_sever_fixed
191875,2021-week_46,AD07F87XC8,Search Million Culture,GB,yes,NaN,Qality,DNNQ,2021,4,1,False
306186,2022-week_03,AD087788D8,MoJoe Brewing Company,IN,yes,NaN,Yield,UserPopY,2022,1,1,False
107956,2021-week_40,AD089RQ89D,Search Million Culture,IN,yes,NaN,Yield,RuleBasedY,2021,4,1,False
115113,2021-week_41,AD0967FZ9Z,Search Million Culture,DE,yes,NaN,Qality,DNNQ,2021,4,1,False
105870,2021-week_40,AD09KKJDDD,Search Million Culture,GB,no,False,Yield,RuleBasedY,2021,4,0,False


In [21]:
three_words_requesters.shape[0]

118141

#### A to Q12:

In [22]:
three_words_requesters[three_words_requesters.ad_id.apply(lambda s: '6' in s)]

,week_id,ad_id,requester,region,is_success,is_sever,recommendation_type,rec_provider,year,quarter,is_success_int,is_sever_fixed
2,2021-week_33,AD08PDP6Y9,Search Million Culture,DE,yes,NaN,Qality,DNNQ,2021,3,1,False
3,2021-week_33,AD89608808,Search Million Culture,DE,yes,NaN,Qality,BooksQ,2021,3,1,False
4,2021-week_33,AD07CMVHP6,Search Million Culture,DE,yes,NaN,Qality,DNNQ,2021,3,1,False
11,2021-week_33,AD88967068,Search Million Culture,DE,yes,NaN,Qality,BooksQ,2021,3,1,False
41,2021-week_33,AD07L6JC8W,MoJoe Brewing Company,DE,yes,NaN,Yield,DNNY,2021,3,1,False
...,...,...,...,...,...,...,...,...,...,...,...,...
373364,2022-week_52,AD06XJR86M,MoJoe Brewing Company,US,yes,NaN,Yield,RuleBasedY,2022,5,1,False
373394,2022-week_52,AD08Y6R9WV,MoJoe Brewing Company,US,yes,NaN,Yield,DNNY,2022,5,1,False
373511,2022-week_52,AD09C68T8Y,MoJoe Brewing Company,US,yes,NaN,Yield,DNNY,2022,5,1,False
373691,2022-week_52,AD08M86WW8,MoJoe Brewing Company,US,yes,NaN,Yield,DNNY,2022,5,1,False


#### A to Q13:

In [23]:
def product_digits(s):
    x=''.join(c for c in s if c.isdigit())
    return math.prod(int(i) for i in x)

In [24]:
three_words_requesters[three_words_requesters.ad_id.apply(product_digits)>30]

,week_id,ad_id,requester,region,is_success,is_sever,recommendation_type,rec_provider,year,quarter,is_success_int,is_sever_fixed
37,2021-week_33,AD88898888,Search Million Culture,DE,yes,NaN,Qality,BooksQ,2021,3,1,False
76,2021-week_33,AD88898786,MoJoe Brewing Company,DE,yes,NaN,Yield,BooksY,2021,3,1,False
112,2021-week_33,AD88788887,Search Million Culture,DE,yes,NaN,Qality,BooksQ,2021,3,1,False
162,2021-week_33,AD88887688,Search Million Culture,DE,yes,NaN,Qality,BooksQ,2021,3,1,False
174,2021-week_33,AD88888876,Search Million Culture,DE,yes,NaN,Qality,BooksQ,2021,3,1,False
...,...,...,...,...,...,...,...,...,...,...,...,...
360315,2022-week_52,AD89888888,MoJoe Brewing Company,GB,yes,NaN,Yield,ManualQ,2022,5,1,False
362075,2022-week_52,AD88888997,Search Million Culture,GB,yes,NaN,Qality,UserPopQ,2022,5,1,False
362199,2022-week_52,AD88688888,Mission Street Manufacturing,GB,no,False,Qality,UserPopQ,2022,5,0,False
363286,2022-week_52,AD88887888,Search Million Culture,GB,yes,NaN,Qality,UserPopQ,2022,5,1,False


### Analysis

1. How many weeks available in the data? Are there any missing weeks? 
1. How many regions available in the data? Are there any missing weeks per region? 
1. How many ads are being sent to annotation in each region per week?
1. How many None values are there in is_sever column? 
1. Are ads being sent more than one time in a given week? what is the ad_id that was sent the most in a given week? (e.g. ad X where sent 50 times in week Y)
1. Are there ads that are being sent in more than one week?
   1. Which is the ad that was sent in most amount of weeks (e.g. ad X where sent in Y different weeks)
   2. What are the weeks that the ad from above was sent in? 
   3. Is there an Ad that was succsefull in one week, but not succesfful in a different week? Show one.
1. When is_sever is None, what is the number of successful requests? What is the number of non-successful requests? What do you learn from it about the reason for Nones in the first place? 
1. Per each region, What is the Quality/Yield traffic proportion WoW? 
1. How many different reqeusters are there? 
1. Which are the top 5 requesters per region? 
1. Which are the different rec_proivders?
1. Are there different rec providers per region? 
1. Are there different rec providers per rec type?
1. What are the notation rules distinguishing between quality vs yielding providers?
1. Which is the most successful region of operation? 
1. Which is the most successful rec provider per each region? 
1. Present a table with a success rate, sever defects rate and CI (for each metric) per region
1. Present a table with a success rate, sever defects rate and CI (for each metric) per rec provider
1. Present a table with a success rate, sever defects rate and CI (for each metric) per region and rec provider combinations
1. Present a table with a success rate, sever defects rate and CI (for each metric) per rec type (Q/Y)
1. Present a table with a success rate, sever defects rate and CI (for each metric) per rec type and region
1. Present WoW table/graph with success rate and CI (see image below) per region
1. Present WoW table/graph with success rate and CI (see image below) per region and rec type
1. Present WoW table/graph with success rate and CI (see image below) per region and rec provider
1. Which are the requester to which we succeed the most?
1. Which are the requester to which we produce the most critical defects?
1. What is the overall success rate trend over time? 
1. What is the overall sever defect rate trend over time? 
1. Preset a WoW graph showing the number of requests per customer in each region (hint: [use stacked bars](https://plotly.com/python/bar-charts/)), from it:
  1. Identify major traffic shape changes (addition/removal) of major clients
1. Preset a WoW graph showing the **requests proportion** per customer in each region (hint: [use stacked bars](https://plotly.com/python/bar-charts/)), from it:
  1. Identify major traffic shape changes (addition/removal) of major clients


#### A to Q1:

In [25]:
sorted(df.week_id.unique())

['2021-week_33',
 '2021-week_34',
 '2021-week_35',
 '2021-week_36',
 '2021-week_37',
 '2021-week_38',
 '2021-week_39',
 '2021-week_40',
 '2021-week_41',
 '2021-week_42',
 '2021-week_43',
 '2021-week_44',
 '2021-week_45',
 '2021-week_46',
 '2021-week_47',
 '2021-week_48',
 '2021-week_49',
 '2021-week_50',
 '2021-week_51',
 '2022-week_01',
 '2022-week_02',
 '2022-week_03',
 '2022-week_04',
 '2022-week_05',
 '2022-week_06',
 '2022-week_52']

In [26]:
df.week_id.value_counts().sort_index()

2021-week_33    14356
2021-week_34    14371
2021-week_35    14373
2021-week_36    14390
2021-week_37    14392
2021-week_38    14347
2021-week_39    14416
2021-week_40    14406
2021-week_41    14380
2021-week_42    14376
2021-week_43    14367
2021-week_44    14299
2021-week_45    14440
2021-week_46    14350
2021-week_47    14402
2021-week_48    14404
2021-week_49    14337
2021-week_50    14404
2021-week_51    12805
2022-week_01    14385
2022-week_02    14423
2022-week_03    14317
2022-week_04    14367
2022-week_05    14362
2022-week_06    14389
2022-week_52    15930
Name: week_id, dtype: int64

#### A to Q2:

Option 1:

In [27]:
df['week_number'] = df.week_id.apply(lambda x: int(x.split('_')[1]))

In [28]:
df.groupby('region').week_number.unique()

region
DE    [33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 4...
GB    [33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 4...
IN    [33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 4...
JP    [33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 4...
US    [33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 4...
Name: week_number, dtype: object

In [29]:
def find_missing_week_id(week_ids):
    missing_weeks = []
    for i in range (33, 53):
        if i not in week_ids:
            missing_weeks.append(i)
    return missing_weeks

In [30]:
df.groupby('region').week_number.unique().apply(find_missing_week_id)

region
DE      []
GB      []
IN      []
JP    [51]
US      []
Name: week_number, dtype: object

Option 2:

In [31]:
row_count_per_week_and_region = df.groupby(['week_id', 'region']).size()
row_count_per_week_and_region.head()

week_id       region
2021-week_33  DE        2395
              GB        3180
              IN        3196
              JP        1611
              US        3974
dtype: int64

In [32]:
row_count_per_week_and_region = row_count_per_week_and_region.unstack('region')
row_count_per_week_and_region.head()

region,DE,GB,IN,JP,US
week_id,,,,,
2021-week_33,2395.0,3180.0,3196.0,1611.0,3974.0
2021-week_34,2360.0,3187.0,3219.0,1599.0,4006.0
2021-week_35,2393.0,3209.0,3209.0,1580.0,3982.0
2021-week_36,2401.0,3209.0,3213.0,1589.0,3978.0
2021-week_37,2404.0,3181.0,3183.0,1644.0,3980.0


In [33]:
row_count_per_week_and_region.sample().iloc[0]

region
DE    2402.0
GB    3191.0
IN    3184.0
JP    1599.0
US    4009.0
Name: 2022-week_01, dtype: float64

In [34]:
def count_nans_in_row(row):
    return row.isnull().sum()

In [35]:
row_count_per_week_and_region.apply(count_nans_in_row, axis=1)

week_id
2021-week_33    0
2021-week_34    0
2021-week_35    0
2021-week_36    0
2021-week_37    0
2021-week_38    0
2021-week_39    0
2021-week_40    0
2021-week_41    0
2021-week_42    0
2021-week_43    0
2021-week_44    0
2021-week_45    0
2021-week_46    0
2021-week_47    0
2021-week_48    0
2021-week_49    0
2021-week_50    0
2021-week_51    1
2022-week_01    0
2022-week_02    0
2022-week_03    0
2022-week_04    0
2022-week_05    0
2022-week_06    0
2022-week_52    0
dtype: int64

In [36]:
count_nans_in_row(row_count_per_week_and_region.iloc[0])

0

In [37]:
count_nans_in_row(pd.Series([None, 1, 2, None]))

2

In [38]:
df.groupby(['week_id', 'region']).size().unstack(['region']).apply(lambda col: 'None' if col.isnull().sum() == 0 else col[col.isnull()].index,axis=0)

region
DE                                                 None
GB                                                 None
IN                                                 None
JP    Index(['2021-week_51'], dtype='object', name='...
US                                                 None
dtype: object

In [39]:
print(f'Number of region available in the data : {df.region.value_counts().shape[0]}')

Number of region available in the data : 5


A to Q3:

In [40]:
df.groupby(['region','week_id']).ad_id.count()

region  week_id     
DE      2021-week_33    2395
        2021-week_34    2360
        2021-week_35    2393
        2021-week_36    2401
        2021-week_37    2404
                        ... 
US      2022-week_03    3979
        2022-week_04    3999
        2022-week_05    3998
        2022-week_06    3992
        2022-week_52    3998
Name: ad_id, Length: 129, dtype: int64

#### A to Q4:

In [41]:
print(f'Number of NaN in is_sever column : {df.is_sever.isna().sum()}')

Number of NaN in is_sever column : 304195


#### A to Q5:

In [42]:
groupby_week=df.groupby('week_id').ad_id.value_counts()
groupby_week[groupby_week>1].sample(5)

max_count_ad_idx=groupby_week.idxmax()
max_count_ad=groupby_week.loc[max_count_ad_idx]
                      
print(f'ad {max_count_ad_idx[1]} where sent {max_count_ad} times at {max_count_ad_idx[0]}')

ad AD098SWYF6 where sent 81 times at 2021-week_40


#### A to Q6:

In [43]:
groupby_ad=df.groupby('ad_id').week_id.count()
ad_max_count_week_idx=groupby_ad.idxmax()
max_count_week=groupby_ad[ad_max_count_week_idx]

print(f'ad {ad_max_count_week_idx} where sent in different {max_count_week} weeks')

ad AD07PFFMP9 where sent in different 247 weeks


In [44]:
df[df.ad_id==ad_max_count_week_idx].week_id

7031      2021-week_33
7032      2021-week_33
20010     2021-week_34
20011     2021-week_34
35214     2021-week_35
              ...     
363533    2022-week_52
363534    2022-week_52
363535    2022-week_52
363536    2022-week_52
365817    2022-week_52
Name: week_id, Length: 247, dtype: object

In [45]:
ad_success=df[df.is_success=='yes']
ad_no_success=df[df.is_success=='no']

print(pd.merge(ad_success, ad_no_success, on=['ad_id'], how='inner')[['ad_id','week_id_x','is_success_x','week_id_y','is_success_y']].head())

       ad_id     week_id_x is_success_x     week_id_y is_success_y
0  AD07H9DVL  2021-week_33          yes  2021-week_40           no
1  AD07H9DVL  2021-week_33          yes  2021-week_41           no
2  AD07H9DVL  2021-week_35          yes  2021-week_40           no
3  AD07H9DVL  2021-week_35          yes  2021-week_41           no
4  AD07H9DVL  2021-week_40          yes  2021-week_40           no


#### A to Q7:

In [46]:
sever_na_success_yes=df[(df.is_sever.isna()) & (df.is_success=='yes')].shape[0]
sever_na_success_no=df[(df.is_sever.isna()) & (df.is_success=='no')].shape[0]

print(f'Number of is_sever=NaN and is_success=yes id : {sever_na_success_yes}')
print(f'Number of is_sever=NaN and is_success=yes id : {sever_na_success_no}')

sever_true_success_no=df[(df.is_sever==True) & (df.is_success=='no')].shape[0]
sever_false_success_no=df[(df.is_sever==False) & (df.is_success=='no')].shape[0]

print(f'Number of is_sever=true  and is_success=no   : {sever_true_success_no}')
print(f'Number of is_sever=false and is_success=no   : {sever_false_success_no}')

Number of is_sever=NaN and is_success=yes id : 304189
Number of is_sever=NaN and is_success=yes id : 6
Number of is_sever=true  and is_success=no   : 44677
Number of is_sever=false and is_success=no   : 24899


A to Q8:

In [47]:
df.groupby(['region','week_number']).recommendation_type.value_counts().unstack().apply(lambda r:r['Qality']/r['Yield'], axis=1).rename('Qality/Yield Ratio')

region  week_number
DE      1              5.388298
        2              5.523161
        3              3.784708
        4              6.232416
        5              4.631206
                         ...   
US      48             0.446087
        49             1.051361
        50             0.877234
        51             0.433286
        52             0.481289
Name: Qality/Yield Ratio, Length: 129, dtype: float64

#### A to Q9:

In [48]:
df.requester.nunique()

36

#### A to Q10:

In [49]:
df.groupby('region').requester.value_counts().groupby(level=0).head(5)

region  requester             
DE      Search Million Culture    23657
        RelayFoods                17313
        MoJoe Brewing Company      8616
        iDreamsky Technology       3542
        SOLOMO365                  2435
GB      Search Million Culture    28002
        RelayFoods                15920
        Fancy                     13678
        MoJoe Brewing Company      9530
        iDreamsky Technology       3919
IN      Puentes Company           29427
        Search Million Culture    21365
        RelayFoods                17352
        MoJoe Brewing Company      7191
        iDreamsky Technology       1995
JP      RelayFoods                24574
        Search Million Culture     7906
        MoJoe Brewing Company      3549
        LocalVox Media             1724
        Tab Solutions              1366
US      Extreme DA                57215
        Fancy                     18668
        Tab Solutions              9790
        Search Million Culture     5610
        S

#### A to Q11:

In [50]:
df.rec_provider.unique()

array(['DNNQ', 'DNNY', 'BooksQ', 'UserPopQ', 'UserPopY', 'RuleBasedY',
       'ManualY', 'ManualQ', 'BooksY', 'XGBQ', 'XGBY', 'RNNQ', 'RNNY',
       'RuleBased', 'UserPopSelectionQ', 'UserPopSelectionY'],
      dtype=object)

#### A to Q12:

In [51]:
df.groupby('region').rec_provider.nunique()

region
DE     9
GB    12
IN     6
JP     9
US    14
Name: rec_provider, dtype: int64

#### A to Q13:

In [52]:
df.groupby('rec_provider').recommendation_type.nunique()

rec_provider
BooksQ               1
BooksY               1
DNNQ                 1
DNNY                 1
ManualQ              2
ManualY              2
RNNQ                 1
RNNY                 1
RuleBased            1
RuleBasedY           1
UserPopQ             1
UserPopSelectionQ    1
UserPopSelectionY    1
UserPopY             1
XGBQ                 1
XGBY                 1
Name: recommendation_type, dtype: int64

In [53]:
df.groupby('rec_provider').recommendation_type.nunique().reset_index().query('recommendation_type != 1')

,rec_provider,recommendation_type
4,ManualQ,2
5,ManualY,2


In [54]:
(df.groupby('rec_provider').recommendation_type!=1)

True

#### A to Q14:

In [55]:
df.groupby(['rec_provider','recommendation_type'])[['rec_provider','recommendation_type']].value_counts()

rec_provider       recommendation_type
BooksQ             Qality                   1720
BooksY             Yield                    4150
DNNQ               Qality                 117424
DNNY               Yield                   45116
ManualQ            Qality                  10220
                   Yield                    3624
ManualY            Qality                   1439
                   Yield                     543
RNNQ               Qality                  20983
RNNY               Yield                   12732
RuleBased          Qality                    182
RuleBasedY         Yield                   28154
UserPopQ           Qality                  69937
UserPopSelectionQ  Qality                   2417
UserPopSelectionY  Yield                      21
UserPopY           Yield                   38600
XGBQ               Qality                  12250
XGBY               Yield                    4276
dtype: int64

In [56]:
df[df.rec_provider.str[-1]!=df.recommendation_type.str[0]][['rec_provider','recommendation_type']]

,rec_provider,recommendation_type
24,ManualY,Qality
313,ManualY,Qality
314,ManualY,Qality
315,ManualY,Qality
316,ManualY,Qality
...,...,...
373203,ManualQ,Yield
373272,ManualY,Qality
373570,ManualQ,Yield
373584,ManualQ,Yield


In [57]:
l1=df.rec_provider.unique()
l2=df.rec_provider.str[:-1].unique()
l1_intersec_l2=[x for x in l1 if x in l2]
print(l1_intersec_l2)

['RuleBased']


In [58]:
df[df.rec_provider=='RuleBased']['recommendation_type'].sample(10)

113268    Qality
91727     Qality
305924    Qality
212785    Qality
313482    Qality
147635    Qality
71664     Qality
289208    Qality
27213     Qality
327038    Qality
Name: recommendation_type, dtype: object

#### A to Q15:

In [59]:
df.groupby('region').is_success.value_counts().unstack()['yes'].idxmax()

'US'

#### A to Q16:

In [60]:
df.groupby(['region','rec_provider']).is_success.value_counts().unstack()['yes'].groupby(level=0).idxmax().str[1]

region
DE        DNNQ
GB        DNNQ
IN    UserPopQ
JP        DNNQ
US        DNNY
Name: yes, dtype: object

#### A to Q17:

In [61]:
def CI_prob(p_hat,n,alpha,item):
    if item == 'low':
        sgn=-1
    if item == 'high':
        sgn=1
    
    z= norm.ppf(1-alpha/2)
    return p_hat+sgn*z*math.sqrt(p_hat*(1-p_hat)/n)

In [62]:
df1=df.groupby('region').is_success.value_counts(normalize=True).unstack('is_success')['yes'].rename('success_rate').to_frame()
df1['count']=df.groupby('region').is_success.count()

df1['CI_95_low']= df1.apply(lambda row: CI_prob(row['success_rate'], row['count'],0.05,'low') ,axis=1)
df1['CI_95_high']=df1.apply(lambda row: CI_prob(row['success_rate'], row['count'],0.05,'high'),axis=1)
print(df1)

        success_rate   count  CI_95_low  CI_95_high
region                                             
DE          0.814192   62177   0.811135    0.817249
GB          0.837619   82984   0.835110    0.840129
IN          0.761900   83154   0.759005    0.764795
JP          0.763927   41627   0.759848    0.768007
US          0.856249  103846   0.854115    0.858383


In [63]:
df2=df.groupby(['region','is_success']).is_sever.value_counts(normalize=True).unstack('is_success')['no'].unstack('is_sever')[True].rename('sever_rate').to_frame()
df2['count']=df.groupby(['region','is_success']).is_sever.value_counts().unstack('is_success')['no'].unstack('is_sever').sum(axis=1)

df2['CI_95_low'] =df2.apply(lambda x:CI_prob(x['sever_rate'],x['count'],0.05,'low') ,axis=1)
df2['CI_95_high']=df2.apply(lambda x:CI_prob(x['sever_rate'],x['count'],0.05,'high'),axis=1)
print(df2)

        sever_rate    count  CI_95_low  CI_95_high
region                                            
DE        0.611255  11550.0   0.602365    0.620145
GB        0.736752  13474.0   0.729316    0.744188
IN        0.592535  19798.0   0.585690    0.599379
JP        0.657032   9826.0   0.647646    0.666418
US        0.636589  14928.0   0.628873    0.644305


#### A to Q18:

In [64]:
df3=df.groupby('rec_provider').is_success.value_counts(normalize=True).unstack('is_success')['yes'].rename('success_rate').to_frame()
df3['count']=df.groupby('rec_provider').is_success.count()

df3['CI_95_low']= df3.apply(lambda row: CI_prob(row['success_rate'], row['count'],0.05,'low') ,axis=1)
df3['CI_95_high']=df3.apply(lambda row: CI_prob(row['success_rate'], row['count'],0.05,'high'),axis=1)
print(df3)

                   success_rate   count  CI_95_low  CI_95_high
rec_provider                                                  
BooksQ                 0.836628    1720   0.819156    0.854100
BooksY                 0.979759    4150   0.975475    0.984044
DNNQ                   0.858376  117424   0.856382    0.860371
DNNY                   0.886714   45116   0.883790    0.889639
ManualQ                0.953193   13844   0.949674    0.956711
ManualY                0.905651    1982   0.892782    0.918520
RNNQ                   0.902540   20983   0.898527    0.906553
RNNY                   0.919651   12732   0.914930    0.924373
RuleBased              0.967033     182   0.941093    0.992973
RuleBasedY             0.574554   28154   0.568779    0.580329
UserPopQ               0.747573   69937   0.744353    0.750792
UserPopSelectionQ      0.815060    2417   0.799582    0.830538
UserPopSelectionY      0.857143      21   0.707479    1.006806
UserPopY               0.726839   38600   0.722394    0

In [65]:
df4=df.groupby(['rec_provider','is_success']).is_sever.value_counts(normalize=True).unstack('is_success')['no'].unstack('is_sever')[True].rename('sever_rate').to_frame()
df4['count']=df.groupby(['rec_provider','is_success']).is_sever.value_counts().unstack('is_success')['no'].unstack('is_sever').sum(axis=1)

df4['CI_95_low'] =df4.apply(lambda x:CI_prob(x['sever_rate'],x['count'],0.05,'low') ,axis=1)
df4['CI_95_high']=df4.apply(lambda x:CI_prob(x['sever_rate'],x['count'],0.05,'high'),axis=1)
print(df4)

                   sever_rate    count  CI_95_low  CI_95_high
rec_provider                                                 
BooksQ               0.099644    281.0   0.064623    0.134665
BooksY               0.261905     84.0   0.167881    0.355928
DNNQ                 0.840640  16629.0   0.835077    0.846203
DNNY                 0.756556   5110.0   0.744789    0.768323
ManualQ              0.526235    648.0   0.487790    0.564679
ManualY              0.529412    187.0   0.457872    0.600951
RNNQ                 0.755501   2045.0   0.736874    0.774129
RNNY                 0.620723   1023.0   0.590990    0.650456
RuleBased            1.000000      6.0   1.000000    1.000000
RuleBasedY           0.526303  11976.0   0.517360    0.535245
UserPopQ             0.475017  17652.0   0.467650    0.482384
UserPopSelectionQ    0.601790    447.0   0.556409    0.647171
UserPopSelectionY    0.333333      3.0  -0.200101    0.866768
UserPopY             0.711969  10544.0   0.703325    0.720613
XGBQ    

#### A to Q19:

In [66]:
df5=df.groupby(['region','rec_provider']).is_success.value_counts(normalize=True).unstack('is_success')['yes'].rename('success_rate').to_frame()
df5['count']=df.groupby(['region','rec_provider']).is_success.count()

df5['CI_95_low'] =df5.apply(lambda x:CI_prob(x['success_rate'],x['count'],0.05,'low') ,axis=1)
df5['CI_95_high']=df5.apply(lambda x:CI_prob(x['success_rate'],x['count'],0.05,'high'),axis=1)
print(df5)

                          success_rate  count  CI_95_low  CI_95_high
region rec_provider                                                 
DE     BooksQ                 0.799704   1353   0.778379    0.821030
       BooksY                 0.966587   1257   0.956652    0.976522
       DNNQ                   0.839114  32433   0.835116    0.843113
       DNNY                   0.917297   5417   0.909963    0.924632
       ManualQ                0.857438    968   0.835413    0.879463
       ManualY                0.960331    605   0.944778    0.975883
       RuleBasedY             0.633209   1614   0.609698    0.656721
       UserPopQ               0.731142  15179   0.724088    0.738195
       UserPopY               0.779469   3351   0.765431    0.793506
GB     DNNQ                   0.874127  39675   0.870863    0.877391
       DNNY                   0.857968   7273   0.849945    0.865991
       ManualQ                0.966574   2154   0.958983    0.974165
       ManualY                0.87

In [67]:
df6=df.groupby(['region','rec_provider','is_success']).is_sever.value_counts(normalize=True).unstack('is_success')['no'].unstack('is_sever')[True].rename('sever_rate').to_frame()
df6['count']=df.groupby(['region','rec_provider','is_success']).is_sever.value_counts().unstack('is_success')['no'].unstack('is_sever').sum(axis=1)

df6['CI_95_low'] =df6.apply(lambda x:CI_prob(x['sever_rate'],x['count'],0.05,'low') ,axis=1)
df6['CI_95_high']=df6.apply(lambda x:CI_prob(x['sever_rate'],x['count'],0.05,'high'),axis=1)
print(df6)

                          sever_rate   count  CI_95_low  CI_95_high
region rec_provider                                                
DE     BooksQ               0.095941   271.0   0.060877    0.131005
       BooksY               0.238095    42.0   0.109285    0.366905
       DNNQ                 0.763703  5218.0   0.752176    0.775229
       DNNY                 0.604027   447.0   0.558690    0.649364
       ManualQ              0.333333   138.0   0.254683    0.411984
       ManualY              1.000000    24.0   1.000000    1.000000
       RuleBasedY           0.292724   591.0   0.256040    0.329408
       UserPopQ             0.498039  4080.0   0.482697    0.513381
       UserPopY             0.668471   739.0   0.634530    0.702412
GB     DNNQ                 0.869017  4993.0   0.859658    0.878375
       DNNY                 0.784124  1033.0   0.759034    0.809213
       ManualQ              0.652778    72.0   0.542809    0.762746
       ManualY              0.657143    70.0   0

#### A to Q20:

In [68]:
df7=df.groupby('recommendation_type').is_success.value_counts(normalize=True).unstack('is_success')['yes'].rename('success_rate').to_frame()
df7['count']=df.groupby('recommendation_type').is_success.count()

df7['CI_95_low'] =df7.apply(lambda x:CI_prob(x['success_rate'],x['count'],0.05,'low') ,axis=1)
df7['CI_95_high']=df7.apply(lambda x:CI_prob(x['success_rate'],x['count'],0.05,'high'),axis=1)
print(df7)

                     success_rate   count  CI_95_low  CI_95_high
recommendation_type                                             
Qality                   0.830910  236572   0.829399    0.832420
Yield                    0.784427  137216   0.782252    0.786603


In [69]:
df8=df.groupby(['recommendation_type','is_success']).is_sever.value_counts(normalize=True).unstack('is_success')['no'].unstack('is_sever')[True].rename('sever_rate').to_frame()
df8['count']=df.groupby(['recommendation_type','is_success']).is_sever.value_counts().unstack('is_success')['no'].unstack('is_sever')[True].to_frame()

df8['CI_95_low'] =df8.apply(lambda x:CI_prob(x['sever_rate'],x['count'],0.05,'low') ,axis=1)
df8['CI_95_high']=df8.apply(lambda x:CI_prob(x['sever_rate'],x['count'],0.05,'high'),axis=1)
print(df8)

                     sever_rate  count  CI_95_low  CI_95_high
recommendation_type                                          
Qality                 0.645766  25830   0.639933    0.651599
Yield                  0.637218  18847   0.630354    0.644082


In [70]:
df.groupby('region').is_success.count()

region
DE     62177
GB     82984
IN     83154
JP     41627
US    103846
Name: is_success, dtype: int64

Analysis Bonus questions:
1. Compute the per region success rate and CI in trailing weeks, Say, 4 weeks instead of 1 week - to allow for smoother estimations and smaller CIs

### Compactization

* Are there duplicated ads? To compac the data, remove duplicated ads and instead add an ad_count column **(pay attention, this is not trivial at all)**

### Merges and joins

The Wow samples and annotations task were sent and recieve seperatly, per each week. The dataset you were working on was constructed from those files. You can see the files under the `data/weekly` folder, here are the first 10 files:

In [71]:
import os
weekly_files = os.listdir('data/weekly/')
sorted(weekly_files)[:10]

['2021_33_DE_Qality_annotation_result.csv',
 '2021_33_DE_Yield_annotation_result.csv',
 '2021_33_DE__Sample.csv',
 '2021_33_GB_Qality_annotation_result.csv',
 '2021_33_GB_Yield_annotation_result.csv',
 '2021_33_GB__Sample.csv',
 '2021_33_IN_Qality_annotation_result.csv',
 '2021_33_IN_Yield_annotation_result.csv',
 '2021_33_IN__Sample.csv',
 '2021_33_JP_Qality_annotation_result.csv']

Your task is to reconstruct the dataset above. 

### Visualizations

Produce the following success rate graph per region:

Produce the following requestser proportion graph:

## Appendix

Code to create weekly files:

In [72]:
# for l in ad_recs_annotated.region.unique():
#     for w in ad_recs_annotated.week_id.unique():
#         w_id = w.split('_')[1]
#         y = w.split('-')[0]
#         query = f'week_id == "{w}" and region == "{l}"'
#         sample_f_name = f'{y}_{w_id}_{l}__Sample.csv'
#         ad_recs_annotated.query(query)[['week_id','ad_id','requester','region','recommendation_type', 'rec_provider']].to_csv(f'data/weekly/{sample_f_name}', index=False)
#         for s in ad_recs_annotated.recommendation_type.unique():
#             ann_f_name = f'{y}_{w_id}_{l}_{s}_annotation_result.csv'
#             query = f'week_id == "{w}" and region == "{l}" and recommendation_type == "{s}"'
#             sss = ad_recs_annotated.query(query)
#             if sss.empty:
#                 continue            
#             sss[['region','ad_id','is_success','is_sever']].drop_duplicates(subset=['ad_id']).to_csv(f'data/weekly/{ann_f_name}', index=False)